In [ ]:
import os
import shutil
import yaml
import time
import json
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
from IPython.display import Image, clear_output
from collections import Counter
from ensemble_boxes import *
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
VER = 'v9'
PARAMS = {
    'version': VER,
    'folds': 5,
    'val_fold': 0,
    'img_size': 640,
    'yolo': 'yolov5x.pt',
    'ytr_img_size': 640,
    'batch_size': 6,
    'epochs': 50,
    'seed': 2020,
    'sup': 'nms', # 'nms' or 'wbf'
    'iou_th': .4,
    'skip_box_th': .0001,
    'comments': ''
}
DATA_PATH = '/u01/mrorange/vinbigdata/data'
WRK_DIR = f'{DATA_PATH}/working/yolo_{VER}'
LBLS_DIR = f'{DATA_PATH}/working/yolo_labels_{VER}'
IMGS_PATH = f'{DATA_PATH}/train_{PARAMS["img_size"]}'
MDLS_PATH = f'/u01/mrorange/vinbigdata/models_{VER}'
YOLO_DIR = f'{DATA_PATH}/working/yolov5'
if not os.path.exists(WRK_DIR):
    os.mkdir(WRK_DIR)
if not os.path.exists(LBLS_DIR):
    os.mkdir(LBLS_DIR)
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)
    
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)

seed_all(PARAMS['seed'])
start_time = time.time()

In [ ]:
label2color = [
    [59,  238, 119], [222, 21,  229], [94,  49,  164], 
    [206, 221, 133], [117, 75,    3], [210, 224, 119], 
    [211, 176, 166], [63,  7,   197], [102, 65,   77], 
    [194, 134, 175], [209, 219,  50], [255, 44,   47], 
    [89,  125, 149], [110, 27,  100]
]

def plot_img(img, size=(18, 18), is_rgb=True, title='', cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title='', cmap='gray', img_size=None):
    rows = len(imgs) // cols + 1
    fig = plt.figure(figsize=(cols * size, rows * size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.axis('off')
    
def draw_bbox(image, box, label, color, thickness=3):   
    alpha = .1
    alpha_box = .4
    overlay_bbox = image.copy()
    overlay_text = image.copy()
    output = image.copy()
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, .6, 1)[0]
    cv2.rectangle(overlay_bbox, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, 
                  (box[0], box[1] - 7 - text_height), 
                  (box[0] + text_width + 2, box[1]),
                  (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.rectangle(output, 
                  (box[0], box[1]), 
                  (box[2], box[3]),
                  color, thickness)
    cv2.putText(output, 
                label.upper(), 
                (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                .6, (255, 255, 255), 1, 
                cv2.LINE_AA)
    return output

In [ ]:
train_df = pd.read_csv(f'{DATA_PATH}/train.csv')
train_df['img_path'] = train_df.apply(lambda row: f'{IMGS_PATH}/{row.image_id}.png', axis =1)
print('train loaded:', train_df.shape)
meta_df = pd.read_csv(f'{DATA_PATH}/train_meta_{PARAMS["img_size"]}.csv')
print('meta loaded:', meta_df.shape)
train_df = pd.merge(train_df, meta_df, on='image_id')
print('merged:', train_df.shape)
train_df.head()

In [ ]:
train_df = train_df[train_df.class_id != 14].reset_index(drop = True)
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
print('classes:', classes)

In [ ]:
train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.dim1, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.dim0, axis =1)
train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.dim1, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.dim0, axis =1)
print(train_df.shape)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

train_df.head()

In [ ]:
imgs = []
for img_id in train_df.sample(n=8)['image_id'].values:
    boxes = train_df.loc[
        train_df['image_id'] == img_id,
        ['x_min', 'y_min', 'x_max', 'y_max']
    ].values
    boxes *= PARAMS['img_size']
    img_labels = train_df.loc[
        train_df['image_id'] == img_id, 
        ['class_id']
    ].values.squeeze()
    path = train_df.loc[
        train_df['image_id'] == img_id,
        ['img_path']
    ].values[0][0]
    img = cv2.imread(path)
    for label_id, box in zip(img_labels, boxes):
        color = label2color[label_id]
        img = draw_bbox(
            img, 
            list(np.int_(box)), 
            classes[label_id], 
            color
        )
    imgs.append(img)

plot_imgs(imgs, size=4, cols=4, cmap=None)

In [ ]:
SHOW = 500
viz_images = []
train_sup = []
for i, img_id in tqdm(enumerate(train_df.image_id.unique()), total=len(train_df.image_id.unique())):
    img_anns = train_df[train_df.image_id == img_id]
    if (i % SHOW == 0):
        boxes_viz = img_anns[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy() * PARAMS['img_size']
        boxes_viz = boxes_viz.tolist() 
        labels_viz = img_anns['class_id'].to_numpy().tolist()
        path = img_anns['img_path'].values[0]
        img  = cv2.imread(path)
        img_ = img.copy()
        for box, label in zip(boxes_viz, labels_viz):
            color = label2color[int(label)]
            img_ = draw_bbox(img_, list(np.int_(box)), classes[label], color)
        viz_images.append(img_)
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []
    boxes_img = []
    labels_img = []
    cls_ids = img_anns['class_id'].unique().tolist()
    count_dict = Counter(img_anns['class_id'].tolist())
    for cid in cls_ids:
        if count_dict[cid] == 1:
            labels_img.append(cid)
            boxes_img.append(
                img_anns[
                    img_anns.class_id == cid
                ][
                    ['x_min', 'y_min', 'x_max', 'y_max']
                ].to_numpy().squeeze().tolist()
            )
        else:
            cls_list =img_anns[img_anns.class_id == cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_anns[
                img_anns.class_id == cid
            ][
                ['x_min', 'y_min', 'x_max', 'y_max']
            ].to_numpy()
            boxes_list.append(bbox.tolist())
            scores_list.append(np.ones(len(cls_list)).tolist())
            weights.append(1)
    if PARAMS['sup'] == 'nms':
        try:
            boxes, scores, box_labels = nms(
                boxes=boxes_list, 
                scores=scores_list, 
                labels=labels_list, 
                weights=weights,
                iou_thr=PARAMS['iou_th']
            )
        except:
            boxes, scores, box_labels = weighted_boxes_fusion(
                boxes_list=boxes_list, 
                scores_list=scores_list,
                labels_list=labels_list, 
                weights=weights,
                iou_thr=PARAMS['iou_th'],
                skip_box_thr=PARAMS['skip_box_th']
            )
    elif PARAMS['sup'] == 'wbf':
        boxes, scores, box_labels = weighted_boxes_fusion(
            boxes_list=boxes_list, 
            scores_list=scores_list,
            labels_list=labels_list, 
            weights=weights,
            iou_thr=PARAMS['iou_th'],
            skip_box_thr=PARAMS['skip_box_th']
        )
    else:
        raise AttributeError('wrong supression param')
    boxes = boxes.tolist()
    box_labels = box_labels.astype(int).tolist()
    boxes.extend(boxes_img)
    box_labels.extend(labels_img)
    for box, label in zip(boxes, box_labels):
        x_min, y_min, x_max, y_max = (box[0], box[1], box[2], box[3])
        train_sup.append(dict(
            image_id=img_id,
            class_name=classes[label],
            class_id=int(label),
            x_min=x_min,
            y_min=y_min,
            x_max=x_max,
            y_max=y_max,
            img_path=img_anns['img_path'].values[0],
            dim0=img_anns['dim0'].values[0],
            dim1=img_anns['dim1'].values[0]
        ))
    if (i % SHOW == 0):
        img__ = img.copy()
        boxes = np.array(boxes) * PARAMS['img_size']
        boxes = boxes.tolist() 
        for box, label in zip(boxes, box_labels):    
            color = label2color[int(label)]
            img__ = draw_bbox(img__, list(np.int_(box)), classes[label], color)
        viz_images.append(img__)

plot_imgs(viz_images, size=6, cols=2, cmap=None)
plt.figtext(.3, .9, 'Original Bboxes', va='top', ha='center', size=12)
plt.figtext(.75, .9, 'WBF', va='top', ha='center', size=12)
plt.show()

In [ ]:
print(train_df.shape)
train_df = pd.DataFrame(train_sup)
print(train_df.shape)
train_df.head()

In [ ]:
train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)
train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)
train_df['area'] = train_df['w'] * train_df['h']
print(train_df.shape)
train_df.head()

In [ ]:
for img_id in tqdm(train_df.image_id.unique().tolist()):
    temp_df = train_df.loc[train_df.image_id == img_id]
    for _, row in temp_df.iterrows():
        with open(f'{LBLS_DIR}/{img_id}.txt', 'a') as file:
            if row.class_id == 14:
                line = ''
            else:
                line = ' '.join([str(x) for x in [
                    row.class_id, row.x_mid, row.y_mid, row.w, row.h
                ]]) + '\n'
            file.write(line)
            
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
features = ['x_min', 'y_min', 'x_max', 'y_max', 'x_mid', 'y_mid', 'w', 'h', 'area']
X = train_df[features]
y = train_df['class_id']
print('X:', X.shape, '| y:', y.shape)

In [ ]:
gkf  = GroupKFold(n_splits=PARAMS['folds'])
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups=train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

In [ ]:
train_files = []
val_files = []
val_files += list(train_df[train_df.fold == PARAMS['val_fold']].img_path.unique())
train_files += list(train_df[train_df.fold != PARAMS['val_fold']].img_path.unique())
len(train_files), len(val_files)

In [ ]:
os.makedirs(f'{WRK_DIR}/labels/train', exist_ok=True)
os.makedirs(f'{WRK_DIR}/labels/val', exist_ok=True)
os.makedirs(f'{WRK_DIR}/images/train', exist_ok=True)
os.makedirs(f'{WRK_DIR}/images/val', exist_ok=True)
for file in tqdm(train_files, desc='train'):
    shutil.copy(file, f'{WRK_DIR}/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(LBLS_DIR, filename + '.txt'), f'{WRK_DIR}/labels/train')
for file in tqdm(val_files, desc='val'):
    shutil.copy(file, f'{WRK_DIR}/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(LBLS_DIR, filename + '.txt'), f'{WRK_DIR}/labels/val')
    
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
with open(f'{WRK_DIR}/train.txt', 'w') as file:
    for path in glob(f'{WRK_DIR}/images/train/*'):
        file.write(path + '\n')
with open(f'{WRK_DIR}/val.txt', 'w') as file:
    for path in glob(f'{WRK_DIR}/images/val/*'):
        file.write(path + '\n')
data = dict(
    train =  f'{WRK_DIR}/train.txt',
    val   =  f'{WRK_DIR}/val.txt',
    nc    = len(classes),
    names = classes
)
with open(f'{WRK_DIR}/vinbigdata.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)
file = open(f'{WRK_DIR}/vinbigdata.yaml', 'r')
print('YAML file:')
print(file.read())

In [ ]:
os.chdir(YOLO_DIR)
print(
    f'using torch: {torch.__version__}',
    f'\ndevice: {torch.cuda.get_device_properties(0) if torch.cuda.is_available() else "CPU"}'
)

In [ ]:
ytr_img_size = PARAMS['ytr_img_size']
ytr_batch = PARAMS['batch_size']
ytr_epochs = PARAMS['epochs']
ytr_data = f'../yolo_{PARAMS["version"]}/vinbigdata.yaml'
ytr_yolo = PARAMS['yolo']

In [ ]:
!/opt/anaconda3/envs/orange/bin/python train.py \
--img $ytr_img_size \
--batch $ytr_batch \
--epochs $ytr_epochs \
--data $ytr_data \
--weights $ytr_yolo \
--cache

In [ ]:
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
!ls -la $YOLO_DIR/runs/train/

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(f'{YOLO_DIR}/runs/train/exp8/results.png')
plt.figure(figsize=(16, 8))
plt.axis('off')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
"""
VER = 'v1'
MDLS_PATH = f'/u01/mrorange/vinbigdata/models_{VER}'
DATA_PATH = '/u01/mrorange/vinbigdata/data'
YOLO_DIR = f'{DATA_PATH}/working/yolov5'
"""
!cp $YOLO_DIR/runs/train/exp8/weights/best.pt $MDLS_PATH